In [1]:
pip install -U ultralytics

  Using cached ultralytics-8.1.47-py3-none-any.whl.metadata (40 kB)
Using cached ultralytics-8.1.47-py3-none-any.whl (750 kB)
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.1.46
    Uninstalling ultralytics-8.1.46:
      Successfully uninstalled ultralytics-8.1.46
Note: you may need to restart the kernel to use updated packages.


In [3]:
import math
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
import cv2

In [4]:
model = YOLO("best.pt")
cap = cv2.VideoCapture("Without Model Implemented.mp4")

In [5]:
!nvidia-smi

Tue Apr 16 23:24:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.01                 Driver Version: 546.01       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8               3W /  50W |      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

out = cv2.VideoWriter('visioneye-distance-calculation.avi', cv2.VideoWriter_fourcc(*'MJPG'), fps, (w, h))

center_point = (0, h)
pixel_per_meter = 10

txt_color, txt_background, bbox_clr = ((0, 0, 0), (255, 255, 255), (255, 0, 255))

while True:
    ret, im0 = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    annotator = Annotator(im0, line_width=2)

    results = model.track(im0, persist=True)
    boxes = results[0].boxes.xyxy.cpu()

    if results[0].boxes.id is not None:
        track_ids = results[0].boxes.id.int().cpu().tolist()

        for box, track_id in zip(boxes, track_ids):
            annotator.box_label(box, label=str(track_id), color=bbox_clr)
            annotator.visioneye(box, center_point)

            x1, y1 = int((box[0] + box[2]) // 2), int((box[1] + box[3]) // 2)    # Bounding box centroid

            distance = (math.sqrt((x1 - center_point[0]) ** 2 + (y1 - center_point[1]) ** 2))/pixel_per_meter

            text_size, _ = cv2.getTextSize(f"Distance: {distance:.2f} m", cv2.FONT_HERSHEY_SIMPLEX,1.2, 3)
            cv2.rectangle(im0, (x1, y1 - text_size[1] - 10),(x1 + text_size[0] + 10, y1), txt_background, -1)
            cv2.putText(im0, f"Distance: {distance:.2f} m",(x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.2,txt_color, 3)

    out.write(im0)
    cv2.imshow("visioneye-distance-calculation", im0)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

out.release()
cap.release()
cv2.destroyAllWindows()

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ---------------------------------------- 1.5/1.5 MB 652.1 kB/s eta 0:00:00
   ---------------------------------------- 2.8/2.8 MB 360.7 kB/s eta 0:00:00

requirements: AutoUpdate success  19.3s, installed 1 package: ['lapx>=0.5.2']
requirements:  Restart runtime or rerun command for updates to take effect


0: 480x640 (no detections), 551.4ms
Speed: 9.1ms preprocess, 551.4ms inference, 2210.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 424.9ms
Speed: 5.0ms preprocess, 424.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 428.9ms
Speed: 2.0ms preprocess, 428.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 438.8ms
Speed: 3.0ms preprocess, 438.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 337.1ms
Speed: 3.0ms prepro